In [ ]:
import numpy as np
import gurobipy as gp
from algorithms import *

Variables to be used to generate nodes and cluster

In [ ]:
np.random.seed(42)

# Number of nodes
n = 200
# Number of clusters
p = 2

capacity_mean = 10
capacity_stddev = 2
weight_mean = 1
weight_stddev = 0.1
lambda_param = 0.5

Generate random nodes, capacities, and weights

In [ ]:
np.random.seed(42)

# Generate 2D nodes and random capacities for clusters
nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
# Generate random weight for each node
weights = generate_weights(n, weight_mean, weight_stddev)

Run CCP - Gurobi clustering algorithm

In [ ]:
clusters, time1, obj_val = solve_model(nodes, capacities, weights, lambda_param,'ccp')

plot_clusters(clusters,nodes)


Run Greedy (Heuristic ) clustering algorithm

In [ ]:
greedy_clusters, time2 = greedy_clustering(nodes, capacities, weights)

plot_clusters(greedy_clusters,nodes)



Calculate In Sample Stability

In [ ]:
insample_stability_score = insample_stability(clusters,nodes, capacities, weights, lambda_param, num_runs=100)
print("Insample Stability:",insample_stability_score)

Calculate Out Sample Stability

In [ ]:
outsample_stability_score = outsample_stability(n, p, weight_mean, weight_stddev, capacity_mean, capacity_stddev, lambda_param, num_runs=10)

print("Outsample Stability:",outsample_stability_score)

Calculate VSS

In [ ]:
num_simulations = 100
np.random.seed(42)
VSS = calculate_VSS(n, p, capacity_mean, capacity_stddev, weight_mean, weight_stddev, lambda_param, num_simulations)
print(f"The Value of Stochastic Solution (VSS) is {VSS:.2f}")